* Criar um banco de dados SQLite com 10 colunas e 20 linhas de dados.
* Conectar o Pyspark a esse banco de dados.
* Realizar algumas análises e filtros nos dados (minímo 3 ações)
* Exportar os resultados para um arquivo Excel.

In [ ]:
!pip install pyspark

In [ ]:
# Importando as bibliotecas

import sqlite3
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [ ]:
# Conexão com banco de dados

conn = sqlite3.connect("base_dados_local.db")
cursor = conn.cursor()

In [ ]:
# Importando a tabela

cursor.execute('''
  CREATE TABLE IF NOT EXISTS tecnologia(
    id_evento INTEGER PRIMARY KEY,
    nome_evento TEXT,
    data_evento TEXT,
    local TEXT,
    tema TEXT,
    publico_alvo TEXT,
    palestrantes INTEGER,
    patrocinadores INTEGER,
    numero_participantes INTEGER,
    orcamento REAL
  )
''')

In [ ]:
cursor.executemany('''
  INSERT INTO tecnologia (nome_evento, data_evento, local, tema, publico_alvo, palestrantes, patrocinadores, numero_participantes, orcamento)
  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''',[
    ('TechSummit','2023-11-15','São Paulo','Inteligência Artificial','Desenvolvedores',5,3,500,12000),
    ('WebDev Conference','2023-11-20','Rio de Janeiro','Desenvolvimento Web','Programadores',8,5,800,15000),
    ('DataScience Meetup','2023-11-25','Belo Horizonte','Ciência de Dados','Analistas de Dados',3,2,300,10000),
    ('Cloud Expo','2023-11-30','Porto Alegre','Computação em Nuvem','Administradores de Sistemas',6,4,600,45000),
    ('Mobile World','2023-12-05','Curitiba','Desenvolvimento Mobile','Desenvolvedores Mobile',7,6,700,30000),
    ('CyberSecurity Forum','2023-12-10','Brasília','Segurança da Informação','Profissionais de Segurança',4,3,400,80000),
    ('AI Workshop','2023-12-15','Salvador','Inteligência Artificial','Estudantes',2,1,200,60000),
    ('FrontEnd Day','2023-12-20','Recife','Desenvolvimento Front-End','Desenvolvedores Front-End',9,7,900,20000),
    ('BigData Summit','2024-01-10','Campinas','Big Data','Engenheiros de Dados',5,4,500,100000),
    ('DevOps Conference','2024-01-15','Goiânia','DevOps','Engenheiros de DevOps',8,6,800,40000),
    ('IoT Expo','2024-01-20','Manaus','Internet das Coisas','Desenvolvedores IoT',6,5,600,30000),
    ('Blockchain Meetup','2024-01-25','Fortaleza','Blockchain','Desenvolvedores Blockchain',3,2,300,9600),
    ('UX/UI Design Week','2024-02-05','Florianópolis','Design UX/UI','Designers UX/UI',7,6,700,15420),
    ('GameDev Conference','2024-02-10','Vitória','Desenvolvimento de Jogos','Desenvolvedores de Jogos',9,8,900,154030),
    ('Robotics Expo','2024-02-15','João Pessoa','Robótica','Engenheiros de Robótica',5,4,500,30000),
    ('Quantum Computing Forum','2024-02-20','Cuiabá','Computação Quântica','Pesquisadores',4,3,400,15350),
    ('GreenTech Summit','2024-03-05','Aracaju','Tecnologia Verde','Empreendedores',6,5,600,10300),
    ('EdTech Conference','2024-03-10','Natal','Tecnologia na Educação','Educadores',8,7,800,5440),
    ('HealthTech Expo','2024-03-15','Macapá','Tecnologia na Saúde','Profissionais de Saúde',7,6,700,70000),
    ('SpaceTech Forum','2024-03-20','Boa Vista','Tecnologia Espacial','Engenheiros Aeroespaciais',5,4,500,50000)
])

In [ ]:
# Salvando as alterações e fechando a conexão

conn.commit() # Salva as alterações

conn.close() # Fecha a conexão

In [ ]:
# Criando a sessão Spark com o driver JDBC
jdbc_driver_path = '/content/sqlite-jdbc.jar'

spark = SparkSession.builder.appName('Análise professional').config('spark.driver.extraClassPath', jdbc_driver_path).getOrCreate()

In [ ]:
# Montando DataFrame

df = spark.read.format("jdbc").options(
    url = "jdbc:sqlite:base_dados_local.db",
    dbtable = "tecnologia",
    driver = "org.sqlite.JDBC"
).load()

df.show(truncate=False)

+---------+-----------------------+-----------+--------------+-------------------------+---------------------------+------------+--------------+--------------------+---------+
|id_evento|nome_evento            |data_evento|local         |tema                     |publico_alvo               |palestrantes|patrocinadores|numero_participantes|orcamento|
+---------+-----------------------+-----------+--------------+-------------------------+---------------------------+------------+--------------+--------------------+---------+
|1        |TechSummit             |2023-11-15 |São Paulo     |Inteligência Artificial  |Desenvolvedores            |5           |3             |500                 |12000.0  |
|2        |WebDev Conference      |2023-11-20 |Rio de Janeiro|Desenvolvimento Web      |Programadores              |8           |5             |800                 |15000.0  |
|3        |DataScience Meetup     |2023-11-25 |Belo Horizonte|Ciência de Dados         |Analistas de Dados         |3   

In [ ]:
#Media de participantes por palestrante
from pyspark.sql.functions import col
df_media_pp = df.select('nome_evento', 'palestrantes', 'numero_participantes')
df_media_pp = df_media_pp.withColumn('media_pessoa_evento', col('numero_participantes') / col('palestrantes'))
df_media_pp.show()

+--------------------+------------+--------------------+-------------------+
|         nome_evento|palestrantes|numero_participantes|media_pessoa_evento|
+--------------------+------------+--------------------+-------------------+
|          TechSummit|           5|                 500|              100.0|
|   WebDev Conference|           8|                 800|              100.0|
|  DataScience Meetup|           3|                 300|              100.0|
|          Cloud Expo|           6|                 600|              100.0|
|        Mobile World|           7|                 700|              100.0|
| CyberSecurity Forum|           4|                 400|              100.0|
|         AI Workshop|           2|                 200|              100.0|
|        FrontEnd Day|           9|                 900|              100.0|
|      BigData Summit|           5|                 500|              100.0|
|   DevOps Conference|           8|                 800|              100.0|

In [ ]:
from pyspark.sql.functions import format_number

#Media de preço do ingresso por participante
df_media_preco = df.select('nome_evento', 'numero_participantes', 'orcamento')

# Calculo para saber a média do preço por participante
df_media_preco = df_media_preco.withColumn('media_preco_palestrante', col('orcamento') / col('numero_participantes'))

# Ordenando os dados do maior para o menor
df_media_preco = df_media_preco.orderBy(col('media_preco_palestrante').desc())

# Formatando as casas decimais para 2
df_media_preco = df_media_preco.withColumn('media_preco_palestrante', format_number(col('media_preco_palestrante'), 2))

df_media_preco.show()

+--------------------+--------------------+---------+-----------------------+
|         nome_evento|numero_participantes|orcamento|media_preco_palestrante|
+--------------------+--------------------+---------+-----------------------+
|         AI Workshop|                 200|  60000.0|                 300.00|
| CyberSecurity Forum|                 400|  80000.0|                 200.00|
|      BigData Summit|                 500| 100000.0|                 200.00|
|  GameDev Conference|                 900| 154030.0|                 171.14|
|     HealthTech Expo|                 700|  70000.0|                 100.00|
|     SpaceTech Forum|                 500|  50000.0|                 100.00|
|          Cloud Expo|                 600|  45000.0|                  75.00|
|       Robotics Expo|                 500|  30000.0|                  60.00|
|   DevOps Conference|                 800|  40000.0|                  50.00|
|            IoT Expo|                 600|  30000.0|           

In [ ]:
from pyspark.sql.functions import col, regexp_extract

# Calcular a contagem de palavras pelo começo
df_publico = df.withColumn('tema_principal_palestra', regexp_extract(col('publico_alvo'), r'^(\w+)', 1))

df_resultado = df_publico.groupBy('tema_principal_palestra',).count().orderBy('count', ascending=False)

df_resultado.show()


+-----------------------+-----+
|tema_principal_palestra|count|
+-----------------------+-----+
|        Desenvolvedores|    6|
|            Engenheiros|    4|
|          Profissionais|    2|
|        Administradores|    1|
|              Designers|    1|
|              Analistas|    1|
|             Educadores|    1|
|         Empreendedores|    1|
|          Pesquisadores|    1|
|             Estudantes|    1|
|          Programadores|    1|
+-----------------------+-----+



In [ ]:
import pandas as pd

with pd.ExcelWriter('Palestra_tecnologia.xlsx') as writer:
  df.toPandas().to_excel(writer, sheet_name='Base Completa', index=False),
  df_media_pp.toPandas().to_excel(writer, sheet_name='Média de pessoas por palestra', index=False),
  df_media_preco.toPandas().to_excel(writer, sheet_name='Preço médio ingressos', index=False),
  df_resultado.toPandas().to_excel(writer, sheet_name='Contagem tema principal', index=False)

In [ ]:
spark.stop()